In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium import plugins as foliumPlugins
import geopandas

In [ ]:
sns.set()

In [ ]:
df = pd.read_csv('p2-arbres-fr.csv', sep=';')
df_save = df.copy()

# 1. Présentation générale du jeu de données

## 1.1. Apperçu des données

In [ ]:
df.head()

## 1.2. Caractéristiques des données

### 1.2.1. Lignes (individus) / Colonnes (variables)

In [ ]:
individus = df.shape[0]
variables = df.shape[1]
print(f'Les données contiennent {individus} lignes et {variables} colonnes')

In [ ]:
columns = df.columns
columns
print('Liste des colonnes: ')
print('- ', end='')
print(*columns, sep = '\n- ')

### 1.2.2. Type des données

In [ ]:
df.dtypes

Pas de problème dans les types de données. Les variables quantitatives sont bien reconnues en tant que valeurs numériques

### 1.2.3. Données dupliquées

In [ ]:
df.loc[df.duplicated(keep=False, subset=['id_emplacement','geo_point_2d_a', 'geo_point_2d_b']),:]

Un arbre est dupliqué (même id emplacement, même geo point). Le premier contenant moins d'informations est supprimé.

In [ ]:
df = df[df.index != 189133]

### 1.2.4. Données manquantes

In [ ]:
df.isna().mean()

In [ ]:
plt.figure(figsize=(10,6))
(df.isna().mean()*100).plot.bar()
plt.title('% des données manquantes par colonne')
plt.ylabel('%')
plt.show()

In [ ]:
plt.figure()
df.groupby(['stade_developpement'], dropna=False).size().plot.pie()
plt.title('Stades de développement')
plt.show()

In [ ]:
plt.figure()
df.groupby(['remarquable'], dropna=False).size().plot.pie()
plt.title('Arbres remarquables')
plt.show()

In [ ]:
df['type_emplacement'].unique()

# 2. Démarche méthodologique d’analyse de données

1. <ins>Néttoyage du jeu de données</ins>
    - Colonnes vides / inutiles / hors sujet / hors compétence
    - Valeurs abérantes
2. <ins>Analyse univariée des colonnes circonference et hauteur</ins>
    - Description, min, max, ecart type, quartiles
    - Histogramme distributions / concentrations
    - Boite à moustache des dispersions

## 2.1. Néttoyage du jeu de données
- La colonne id est inutile pour l'analyse et sera supprimée
- La colonne "numero" est vide elle sera exclue
- La colonne "type_emplacement" ne possède qu'une même valeur unique pour tous les individus: "Arbre". Elle n'a aucune utilité et sera supprimée
- Les colonnes "geo_point_2d_" seront exclues du jeu de données car je n'ai pas de connaissance dans ce domaine
- Les colonnes "complement_adresse" et "variete" sont très vides avec un taux respectif de seulement 15%, 19% de données remplies. Elles seront supprimées
- La colonne "remarquable" n'est remplie qu'à 69% et cette donnée binaire n'est pas imputable, elles sera supprimée.
- La colonne "stade_developpement" est aussi assez vide avec un taux respectif de 67% de données remplies. Cette donnée catégorielle est peut-être imputable en cherchant par apprentissage (kmeans) avec les variables diametre/espece (hors sujet de cette etude). Cette colonne sera supprimée
- Les colonnes "domanialite", "libelle_francais", "genre" et "espece" contiennent quelques valeurs vides (< 0.8%), non imputables, les lignes seront supprimées en fonction de l'analyse faite.<br> Exemple: si l'analyse est bivariée et concerne le diamètre par domanialité, alors les lignes dont la domanialité n'est pas connues seront supprimées)

In [ ]:
df.drop(['id', 'numero', 'type_emplacement', 'geo_point_2d_a', 'geo_point_2d_b', 'complement_addresse', 'variete', 'remarquable', 'stade_developpement'], axis=1, inplace=True)

In [ ]:
df.describe()

Pour les 2 colonnes les valeurs min et max sont abérantes (circonference et hauteur nulle, circonference de 250m et hauteur de 881km)
Avec un peu de recherche (https://www.unjourdeplusaparis.com/paris-vert/arbres-remarquables-paris) on apprend que pour Paris

- La circonférence du plus gros arbre est de 7m
- La hauteur du plus grand arbre est 35m

On pourrait utiliser ces valeurs, mais je vais filtrer mon jeu de données en utilisant les quartiles

In [ ]:
circonference_min = df['circonference_cm'].quantile(0.25)  - 1.5 * (df['circonference_cm'].quantile(0.75) - df['circonference_cm'].quantile(0.25))
circonference_max = df['circonference_cm'].quantile(0.75)  + 1.5 * (df['circonference_cm'].quantile(0.75) - df['circonference_cm'].quantile(0.25))

hauteur_min = df['hauteur_m'].quantile(0.25)  - 1.5 * (df['hauteur_m'].quantile(0.75) - df['hauteur_m'].quantile(0.25))
hauteur_max = df['hauteur_m'].quantile(0.75)  + 1.5 * (df['hauteur_m'].quantile(0.75) - df['hauteur_m'].quantile(0.25))

filtre_circonference = (df['circonference_cm'] > 0) & (df['circonference_cm'] > circonference_min) & (df['circonference_cm'] < circonference_max)
filtre_hauteur = (df['hauteur_m'] > 0) & (df['hauteur_m'] > hauteur_min) & (df['hauteur_m'] < hauteur_max)

Liste de arbres dont la circonférence est considérée abérantes (=0 ou au-dessous de  Q1−1.5IQ ou au-dessus de Q3+1.5IQ)

In [ ]:
df.loc[~filtre_circonference, :].sort_values('circonference_cm', ascending=False)

Liste des arbres dont la hauteur est considérée comme abérante (=0 ou au-dessous de  Q1−1.5IQ ou au-dessus de Q3+1.5IQ)

In [ ]:
df.loc[~filtre_hauteur, :].sort_values('hauteur_m', ascending=False)

## 2.2. Analyse univariée des colonnes circonference et hauteur

In [ ]:
df_circonference_filtered = df.loc[filtre_circonference, :]
df_circonference_filtered['circonference_cm'].describe()

In [ ]:
df_hauteur_filtered = df.loc[filtre_hauteur, :]
df_hauteur_filtered['hauteur_m'].describe()

In [ ]:
# Histogrammes circonferences et hauteurs
plt.figure(figsize=(10,5))
medianprops = {'color':"black"}
meanprops = {'marker':'o', 'markeredgecolor':'black',
            'markerfacecolor':'firebrick'}

circonference_skew = df_circonference_filtered['circonference_cm'].skew()
circonference_kurtosis = df_circonference_filtered['circonference_cm'].kurtosis()
hauteur_skew = df_hauteur_filtered['hauteur_m'].skew()
hauteur_kurtosis = df_hauteur_filtered['hauteur_m'].kurtosis()

plt.subplot(2,1,1)
plt.title('Repartition')
plt.hist(df_circonference_filtered['circonference_cm'].values, bins=20)
plt.ylabel('Nb Individus')
plt.xlabel('Circonférence(cm)')
plt.text(180,30000,f'skew: {circonference_skew:0.2f}, kurtosis: {circonference_kurtosis:0.2f}')

plt.subplot(2,1,2)
plt.hist(df_hauteur_filtered['hauteur_m'].values, bins=20)
plt.ylabel('Nb Individus')
plt.xlabel('Hauteur(m)')
plt.subplots_adjust(hspace=0.5)
plt.text(16.5,28000,f'skew: {hauteur_skew:0.2f}, kurtosis: {hauteur_kurtosis:0.2f}')
plt.show()

- La distribution de la circonférence est étalée à droite avec une concentration légèrement moindre qu'une distribution normale 
- La distribution de la répartition est étalée aussi à droite avec une concentration beaucoup moins accentuée qu'une distribution normale 

In [ ]:
# Box plots circonferences et hauteurs
plt.figure(figsize=(10,4))
medianprops = {'color':"black"}
meanprops = {'marker':'o', 'markeredgecolor':'black',
            'markerfacecolor':'firebrick'}

circonference_mean = df_circonference_filtered['circonference_cm'].mean()
circonference_median = df_circonference_filtered['circonference_cm'].median()
hauteur_mean = df_hauteur_filtered['hauteur_m'].mean()
hauteur_median = df_hauteur_filtered['hauteur_m'].median()

plt.subplot(2,1,1)
plt.title('Dispersions')
plt.boxplot(df_circonference_filtered['circonference_cm'].values, labels=['Circonférence (cm)'], showfliers=False, meanprops=meanprops, vert=False, patch_artist=True, showmeans=True)
plt.text(50,0.6,f'médiane: {circonference_median:0.1f}, moyenne: {circonference_mean:0.1f}, ')
plt.subplot(2,1,2)
plt.boxplot(df_hauteur_filtered['hauteur_m'].values, labels=['Hauteur (m)'], showfliers=False, meanprops=meanprops, vert=False, patch_artist=True, showmeans=True)
plt.text(6,0.6,f'médiane: {hauteur_median:0.1f}, moyenne: {hauteur_mean:0.1f}')
plt.subplots_adjust(hspace=0.5)
plt.show()

# 3. Synthèse de l'analyse de données

# BONUS: Quelques dispersions et répartitions

## 1. Circonférence en fonction de la hauteur

In [ ]:
df_c_h = df.loc[filtre_circonference & filtre_hauteur, ['circonference_cm', 'hauteur_m']]
plt.figure(figsize=(10,6))
plt.scatter(df_c_h['hauteur_m'], df_c_h['circonference_cm'], marker='+', s=5, linewidths=0.5)
plt.title('Dispersion des circonférences en fonction des hauteurs')
plt.xlabel('Hauteur (m)')
plt.ylabel('Circonférence (cm)')
plt.show()

## 2. Quantité par domanialité

In [ ]:
# Repartition
rep_domanialite = df.groupby(['domanialite'], dropna=True).size().sort_values(ascending=False)

# Graph
plt.figure(figsize=(8,4))
bar = plt.bar(rep_domanialite.index, rep_domanialite.values)
plt.xticks(rotation=90)
plt.title('Répartition des arbres par domanialité')
plt.xticks(fontsize=8)
plt.yticks(fontsize=8)
plt.grid(axis='x')
for rect in bar:
    height = rect.get_height()
    plt.text(rect.get_x() + rect.get_width() / 2.0, height, f'{height/len(df)*100:.0f}%', ha='center', va='bottom', fontsize=8)

Plus de la moitié des arbres sont en Alignement et un presque un quart de ceux-ci dans les jardins.

## 3. Quantité par Arrondissement

In [ ]:
# Répartition
rep_arrondissement = df.groupby(['arrondissement'], dropna=True).size().sort_values(ascending=False)

# Graph
sns.set()
plt.figure(figsize=(8,4))
bar = plt.bar(rep_arrondissement.index, rep_arrondissement.values)
plt.xticks(rotation=90)
plt.title('Répartition des arbres par arrondissement')
plt.xticks(fontsize=8)
plt.yticks(fontsize=8)
plt.grid(axis='x')
for rect in bar:
    height = rect.get_height()
    plt.text(rect.get_x() + rect.get_width() / 2.0, height, f'{height/len(df)*100:.0f}%', ha='center', va='bottom', fontsize=8)

Paris 15ième arrive en tête avec 9% des arbres de la ville

## 4. Les 10 espèces les plus présentes

In [ ]:
# Répartition
rep_libelle_francais_top_10 = df.groupby(['libelle_francais'], dropna=True).size().sort_values(ascending=False).head(10)

# Graph
plt.figure(figsize=(8,4))
bar = plt.bar(rep_libelle_francais_top_10.index, rep_libelle_francais_top_10.values)
plt.xticks(rotation=90)
plt.title('Les 10 espèces les plus présentes')
plt.xticks(fontsize=8)
plt.yticks(fontsize=8)
plt.grid(axis='x')
for rect in bar:
    height = rect.get_height()
    plt.text(rect.get_x() + rect.get_width() / 2.0, height, f'{height/len(df)*100:.0f}%', ha='center', va='bottom', fontsize=8)

Les platanes sont largement en tête avec 21%. Les maronniers, tilleuils et érables sont aussi fortement représentés

## 5. Affichage des arbres sur la carte

**ATTENTION L'EXECUTION ET L'AFFICHAGE PEUVENT PRENDRE PLUSIEURS MINUTES**

In [ ]:
# Centrage map sur Paris
paris_map = folium.Map(location = [48.85, 2.32], zoom_start = 12)

# Creation du cluster de markers
clustered_map = foliumPlugins.MarkerCluster().add_to(paris_map)

# Ajouts des points geo2d au cluster
for lat, lng, label, in zip(df_save['geo_point_2d_a'], df_save['geo_point_2d_b'], df_save['lieu']):
    folium.Marker(
        location=[lat, lng],
        icon=None,
         popup=label,
     ).add_to(clustered_map)

# Affichage map
paris_map

## 6. Classement des arbres par distance

In [ ]:
# Creation du GeoDataFrame
gf = df_save.copy()
gf = geopandas.GeoDataFrame(gf, geometry=geopandas.GeoSeries.from_xy(gf['geo_point_2d_b'], gf['geo_point_2d_a'], crs="EPSG:4326"))
gf.head()

In [ ]:
# Ajoute de la distance (arbitrairement à partir 1er de la liste)
gf['distances'] = gf['geometry'].distance(gf['geometry'][0])
gf.head()